# Homework 3


In [6]:
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
from src.cache import Cache, CacheManager

dataset_path = os.getenv('TASK_DATASET_FILE_PATH')
cache = CacheManager(dataset_path).get_cache()


In [7]:
from src.utils import item_cross_trx
min_support = 0.001
cross_trx = item_cross_trx(cache.transactions, min_support=min_support)
cross_trx

cod_prod,1000344,1000926,1001207,1001228,1002407,1003412,1003457,1003474,1003475,1003516,...,1101884,1101886,1102047,1102085,1102152,1102408,1102506,1104570,1104784,3000109
row_0,,,,,,,,,,,,,,,,,,,,,
49074056,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49074665,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49074722,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49075078,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
49075268,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68465108,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
68476458,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
68476461,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
from src.frequent_patterns import FpGrowthAlgorithm, AprioriAlgorithm

apriori = AprioriAlgorithm(cross_trx).run(min_support = min_support)
apriori.df

,support,itemsets,apriori
0,0.018762,(1000344),True
1,0.016132,(1000926),True
2,0.050009,(1001207),True
3,0.018441,(1001228),True
4,0.025773,(1002407),True
...,...,...,...
1666,0.001414,"(1101882, 1101884, 1102085)",True
1667,0.001009,"(1102408, 1101882, 1101884)",True
1668,0.002408,"(1101882, 1102085, 1101886)",True
1669,0.001521,"(1102408, 1101882, 1101886)",True


In [9]:
fpgrowth = FpGrowthAlgorithm(cross_trx).run(min_support = min_support)
fpgrowth.df

,support,itemsets,fpgrowth
0,0.403269,(1090011),True
1,0.091715,(1028794),True
2,0.044826,(1021437),True
3,0.032409,(1008775),True
4,0.050009,(1001207),True
...,...,...,...
1666,0.001636,"(1003457, 1037365)",True
1667,0.001246,"(1099668, 1037365)",True
1668,0.001200,"(1082480, 1037365)",True
1669,0.001116,"(1037365, 1001207)",True


In [10]:
compare_df = fpgrowth.compare(apriori)
compare_df

,support_fpgrowth,itemsets,fpgrowth,support_apriori,apriori
0,0.403269,(1090011),True,0.403269,True
1,0.001216,"(1021437, 1012174)",True,0.001216,True
2,0.004503,"(1012222, 1012174)",True,0.004503,True
3,0.001927,"(1028794, 1012174)",True,0.001927,True
4,0.006078,"(1101882, 1012174)",True,0.006078,True
...,...,...,...,...,...
1666,0.002309,"(1008354, 1008775)",True,0.002309,True
1667,0.001476,"(1090346, 1008354)",True,0.001476,True
1668,0.001468,"(1008354, 1012141)",True,0.001468,True
1669,0.001101,"(1102085, 1102047)",True,0.001101,True
